# Машинное обучение для текстов. Проект

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

Данный проект также выполнен локально с BERT и Catboost text_features:
- BERT: https://drive.google.com/file/d/1sIqWh-Hty9FOBba7GqbMcy21je8xsof_/view?usp=sharing
- CatBoost: https://drive.google.com/file/d/1V31q_SRPvgMzkbQlpDoyjr-_aE6SwX9s/view?usp=sharing



**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [3]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Подготовим признаки и целевой признак перед обучением и разделим выборки.
Создадим корпус текстов и вычислим для него TF-IDF с помощью TfidfVectorizer()

In [4]:
%%time

m = Mystem()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

df['lemm_text'] = df['text'].apply(lemmatize_text)

df = df.drop(['text'], axis=1)
del m

CPU times: user 1min 9s, sys: 12.6 s, total: 1min 21s
Wall time: 2min 56s


In [5]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=12082020)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=12082020)

In [6]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values.astype('U'))
features_valid = count_tf_idf.transform(features_valid['lemm_text'].values.astype('U'))
features_test = count_tf_idf.transform(features_test['lemm_text'].values.astype('U'))
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(95742, 126292)
(31914, 126292)
(31915, 126292)


## Обучение

Для обучения выберем LogisticRegression

In [11]:
%%time

classificator = LogisticRegression()
hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10],
                'class_weight':['balanced', 'none']}]


print('# Tuning hyper-parameters for f1_score')
print()
clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=3)
clf.fit(features_train, target_train)
print("Best parameters set found on development set:")
print()
LR_best_params = clf.best_params_
print(LR_best_params)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
print()

cv_f1_LR = max(means)

# Tuning hyper-parameters for f1_score

Best parameters set found on development set:

{'C': 10, 'class_weight': 'balanced', 'solver': 'liblinear'}

Grid scores on development set:

0.701968 for {'C': 0.1, 'class_weight': 'balanced', 'solver': 'newton-cg'}
0.701968 for {'C': 0.1, 'class_weight': 'balanced', 'solver': 'lbfgs'}
0.701905 for {'C': 0.1, 'class_weight': 'balanced', 'solver': 'liblinear'}
0.361926 for {'C': 0.1, 'class_weight': 'none', 'solver': 'newton-cg'}
0.361790 for {'C': 0.1, 'class_weight': 'none', 'solver': 'lbfgs'}
nan for {'C': 0.1, 'class_weight': 'none', 'solver': 'liblinear'}
0.746687 for {'C': 1, 'class_weight': 'balanced', 'solver': 'newton-cg'}
0.746687 for {'C': 1, 'class_weight': 'balanced', 'solver': 'lbfgs'}
0.746746 for {'C': 1, 'class_weight': 'balanced', 'solver': 'liblinear'}
0.673228 for {'C': 1, 'class_weight': 'none', 'solver': 'newton-cg'}
0.673228 for {'C': 1, 'class_weight': 'none', 'solver': 'lbfgs'}
nan for {'C': 1, 'class_weight': 'none', 'so

In [12]:
%%time

classificator = LogisticRegression()
classificator.set_params(**LR_best_params)
classificator.fit(features_train, target_train)
target_predict = classificator.predict(features_valid)
valid_f1_LR = f1_score(target_valid, target_predict)
print('F1 на cv', cv_f1_LR)
print('F1 на валидации', valid_f1_LR)

F1 на cv 0.7639381490898353
F1 на валидации 0.7675818528850389
CPU times: user 14 s, sys: 13.2 s, total: 27.3 s
Wall time: 27.3 s


Проверим на тестовой выборке

In [15]:
classificator = LogisticRegression()
classificator.set_params(**LR_best_params)
classificator.fit(features_train, target_train)
predict_test = classificator.predict(features_test)
print('F1:', f1_score(target_test, predict_test))

F1: 0.7614080834419819


Итоговые F1 на других моделях:
- LR с BERT: https://drive.google.com/file/d/1sIqWh-Hty9FOBba7GqbMcy21je8xsof_/view?usp=sharing
- CatBoost: https://drive.google.com/file/d/1V31q_SRPvgMzkbQlpDoyjr-_aE6SwX9s/view?usp=sharing

## Выводы

  Результаты проверки обученных моделей на тестовой выборке:
  - LogisticRegression на подготовленных признаках с помощью TF-IDF: **0.761**
  - LogisticRegression на подготовленных признаках с помощью BERT: **0.722**
  - CatBoost на подготовленных признаках с помощью text_features: **0.783**

Следует отметить, что подготовка признаков с помощью BERT занимает на локальной видеокарте ~1,5 часа, на видеокарте Colab ~3 часа. В случаях LogisticRegression TF-IDF вычисление проекта заняло ~30 мин, а CatBoost text_features - всего ~5 мин.

**Вывод** - лучшая модель CatBoost на подготовленных признаках с помощью text_features